<a href="https://colab.research.google.com/github/kate-melnykova/BCdata2018-SNC-Lavalin-project/blob/main/DataCleaning_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
import folium
%matplotlib inline

from os import path, getcwd
from glob import glob
import pykalman
import datetime as dt
from multiprocessing import Pool

In [ ]:
data_dir = '../data/workshop-content18/3-snc/data/'
ais_pathnames = glob(data_dir + '*.txt')
ais_basenames = [path.basename(pn) for pn in ais_pathnames]
for bn in ais_basenames:
    print(bn)

PMV_AIS_NewWestminster_Current.txt
PMV_AIS_Deltaport_Current.txt
PMV_AIS_Deltaport_History.txt
PMV_AIS_NewWestminster_History.txt


In [ ]:
delta_cur_basename = next(bn for bn in ais_basenames if 'Deltaport_Current' in bn)
delta_his_basename = next(bn for bn in ais_basenames if 'Deltaport_History' in bn)

In [ ]:
#max_rows=int(3e7)
delta_cur = pd.read_csv(
    data_dir + delta_cur_basename, sep='\t',
    parse_dates=['ReceivedTime'],usecols=['UserID','ROT','SOG','Longitude','Latitude',
                                          'TrueHeading','ReceivedTime'])

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
delta_cur.head()

,UserID,ROT,SOG,Longitude,Latitude,TrueHeading,ReceivedTime
0,316005621,-127.0,0.1,-122.77156,49.23065,511.0,2013-10-22 01:05:24.510
1,316018851,0.0,1.7,-123.05445,49.29853,110.0,2013-10-22 01:05:25.400
2,316003679,-127.0,0.1,-123.10751,49.31308,511.0,2013-10-22 01:05:25.853
3,316014621,127.0,12.3,-123.09534,49.29940,210.0,2013-10-22 01:05:26.027
4,316005721,-127.0,0.1,-123.10684,49.31094,511.0,2013-10-22 01:05:26.620


In [ ]:
delta_cur = delta_cur.dropna(axis=0)
delta_cur.head()

,UserID,ROT,SOG,Longitude,Latitude,TrueHeading,ReceivedTime
0,316005621,-127.0,0.1,-122.77156,49.23065,511.0,2013-10-22 01:05:24.510
1,316018851,0.0,1.7,-123.05445,49.29853,110.0,2013-10-22 01:05:25.400
2,316003679,-127.0,0.1,-123.10751,49.31308,511.0,2013-10-22 01:05:25.853
3,316014621,127.0,12.3,-123.09534,49.29940,210.0,2013-10-22 01:05:26.027
4,316005721,-127.0,0.1,-123.10684,49.31094,511.0,2013-10-22 01:05:26.620


In [ ]:
if delta_cur.isna().iloc[-1,:].sum() > 10:
    delta_cur = delta_cur.iloc[:-1, :]

print('# na:', (delta_cur.isna().sum(axis=1) > 5).sum())

# na: 0


In [ ]:
delta_cur.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2560654 entries, 0 to 2560653
Data columns (total 7 columns):
UserID          object
ROT             float64
SOG             float64
Longitude       float64
Latitude        float64
TrueHeading     float64
ReceivedTime    datetime64[ns]
dtypes: datetime64[ns](1), float64(5), object(1)
memory usage: 236.3+ MB


In [ ]:
#extract only lat and lon needed
mask = (delta_cur['Longitude']>-127)&(delta_cur['Longitude']<-120)
mask = mask & (delta_cur['Latitude']>48)|(delta_cur['Latitude']<50)

delta_cur = delta_cur[mask]

In [ ]:
# 6 minutes = 360 seconds
delta_cur['Time2'] = delta_cur['ReceivedTime'].apply(lambda x: int(360*round(x.timestamp()/360)))

In [ ]:
#delta_cur = delta_cur.drop(['ReceivedTime'],axis=1)

In [ ]:
delta_cur.head()

,UserID,ROT,SOG,Longitude,Latitude,TrueHeading,ReceivedTime,Time2
0,316005621,-127.0,0.1,-122.77156,49.23065,511.0,2013-10-22 01:05:24.510,1382403960
1,316018851,0.0,1.7,-123.05445,49.29853,110.0,2013-10-22 01:05:25.400,1382403960
2,316003679,-127.0,0.1,-123.10751,49.31308,511.0,2013-10-22 01:05:25.853,1382403960
3,316014621,127.0,12.3,-123.09534,49.29940,210.0,2013-10-22 01:05:26.027,1382403960
4,316005721,-127.0,0.1,-123.10684,49.31094,511.0,2013-10-22 01:05:26.620,1382403960


In [ ]:
delta_cur.shape

(2502883, 8)

In [ ]:
nPoints_by_UserID = pd.value_counts(delta_cur.UserID)
nPoints_threshold = 1000
print('num rows for which userid count <= 100:', nPoints_by_UserID.loc[nPoints_by_UserID <= 100].values.sum())
userID_allowed = nPoints_by_UserID.index[nPoints_by_UserID > nPoints_threshold]

num rows for which userid count <= 100: 4475


In [ ]:
#number of ships selected
len(list(userID_allowed))

266

In [ ]:
delta_cur = delta_cur[delta_cur['UserID'].isin(list(userID_allowed))]

In [ ]:
delta_cur.head()

,UserID,ROT,SOG,Longitude,Latitude,TrueHeading,ReceivedTime,Time2
0,316005621,-127.0,0.1,-122.77156,49.23065,511.0,2013-10-22 01:05:24.510,1382403960
1,316018851,0.0,1.7,-123.05445,49.29853,110.0,2013-10-22 01:05:25.400,1382403960
2,316003679,-127.0,0.1,-123.10751,49.31308,511.0,2013-10-22 01:05:25.853,1382403960
3,316014621,127.0,12.3,-123.09534,49.29940,210.0,2013-10-22 01:05:26.027,1382403960
4,316005721,-127.0,0.1,-123.10684,49.31094,511.0,2013-10-22 01:05:26.620,1382403960


In [ ]:
df = delta_cur.copy()
#process time
path_id = 0
# figure out if the ship leaves the Metro Vancouver
time_diff = np.diff(df['Time2']) #the time between subsequent signals
col_names = df.columns.values.tolist()
col_names.append('PathID')
dfs = [] #here we store dataframe each of them is a part of unique path
idx = 0
# 1. invesgate all cases where the ship was not tracked for an hour or more.
#If ship is near the edge of Metro Vancouver, split path into two parts
#-- we assume that it leaves the Metro Vancouver Area
# 2. if no data is taken for 2 hours, we split the path into two paths
# 3. if a path contains less than 10 datapoints, drop it
for i in range(len(time_diff)):
    if time_diff[i]>7200:
        df_temp = df.iloc[idx:(i+1)]
        df_temp['PathID'] = path_id
        dfs.append(df_temp)
        idx = i+1
        path_id += 1

    elif time_diff[i]>3600: #1 hour = 60*60=3600 seconds
        #investigate where the ship is located
        lat = df.loc[i,'Latitude']
        lon = df.loc[i,'Longitude']
        #recall original filtering
        #mask = (delta_cur['Longitude']>-127)&(delta_cur['Longitude']<-120)
        #mask = mask & (delta_cur['Latitude']>48)|(delta_cur['Latitude']<50)
        if (lon>-126.8)&(lon<-119.8)&(lat>48.2)&(lat<49.8):
            #the ship is inside Metro Vancouver
            continue
        df_temp = df.iloc[idx:(i+1)]
        df_temp['Path_id'] = path_id
        dfs.append(df_temp)
        idx = i+1
        path_id +=1
           
dfs.append(df.loc[idx:])  #append whatever chunk contains the last row          

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [ ]:
dfs[0]

,UserID,ROT,SOG,Longitude,Latitude,TrueHeading,ReceivedTime,Time2,PathID
0,316005621,-127.0,0.1,-122.77156,49.23065,511.0,2013-10-22 01:05:24.510,1382403960,0
1,316018851,0.0,1.7,-123.05445,49.29853,110.0,2013-10-22 01:05:25.400,1382403960,0
2,316003679,-127.0,0.1,-123.10751,49.31308,511.0,2013-10-22 01:05:25.853,1382403960,0
3,316014621,127.0,12.3,-123.09534,49.29940,210.0,2013-10-22 01:05:26.027,1382403960,0
4,316005721,-127.0,0.1,-123.10684,49.31094,511.0,2013-10-22 01:05:26.620,1382403960,0
5,316014803,-127.0,0.0,-123.05973,49.29095,511.0,2013-10-22 01:05:26.823,1382403960,0
6,316014995,0.0,0.0,-123.14794,49.02514,211.0,2013-10-22 01:05:26.933,1382403960,0
7,316001245,0.0,0.0,-123.08465,49.12455,65.0,2013-10-22 01:05:27.993,1382403960,0
8,316006154,-126.0,1.3,-123.46792,49.47043,3.0,2013-10-22 01:05:28.120,1382403960,0
9,316005622,-127.0,0.1,-123.00487,49.15575,511.0,2013-10-22 01:05:28.260,1382403960,0


In [ ]:
delta_cur.head()

,UserID,ROT,SOG,Longitude,Latitude,TrueHeading,ReceivedTime,Time2
0,316005621,-127.0,0.1,-122.77156,49.23065,511.0,2013-10-22 01:05:24.510,1382403960
1,316018851,0.0,1.7,-123.05445,49.29853,110.0,2013-10-22 01:05:25.400,1382403960
2,316003679,-127.0,0.1,-123.10751,49.31308,511.0,2013-10-22 01:05:25.853,1382403960
3,316014621,127.0,12.3,-123.09534,49.29940,210.0,2013-10-22 01:05:26.027,1382403960
4,316005721,-127.0,0.1,-123.10684,49.31094,511.0,2013-10-22 01:05:26.620,1382403960


# Summarizing all processing

In [ ]:
def find_paths(df_ship):
    col_names = df_ship.columns.values.tolist()
    col_names.append('PathID')
    df_new = pd.DataFrame(columns = col_names)
    
    df_ship = df_ship.reset_index(drop=True)
    time_diff = np.diff(df_ship['Time2']) #the time between subsequent signals
    idx = 0 #index of the beginning of the path
    path_id = 1
    # 1. invesgate all cases where the ship was not tracked for an hour or more.
    #If ship is near the edge of Metro Vancouver, split path into two parts
    #-- we assume that it leaves the Metro Vancouver Area
    # 2. if no data is taken for 2 hours, we split the path into two paths
    for i in range(len(time_diff)):
        if time_diff[i]>7200:
            df_temp = df_ship.iloc[idx:(i+1)].copy()
            df_temp['PathID'] = path_id
            df_new = df_new.append(df_temp,ignore_index=True)
            idx = i + 1
            path_id += 1
        elif time_diff[i] > 3600: #1 hour = 60*60=3600 seconds
            #investigate where the ship is located
            lat = df_ship.loc[i,'Latitude']
            lon = df_ship.loc[i,'Longitude']
            #recall original filtering
            #mask = (delta_cur['Longitude']>-127)&(delta_cur['Longitude']<-120)
            #mask = mask & (delta_cur['Latitude']>48)|(delta_cur['Latitude']<50)
            if (lon>-126.8)&(lon<-119.8)&(lat>48.2)&(lat<49.8):
                #the ship is inside Metro Vancouver
                df_temp = df_ship.iloc[idx:(i+1)].copy()
                df_temp['PathID'] = path_id
                df_new = df_new.append(df_temp,ignore_index=True)
                idx = i+1
                path_id += 1
    df_temp = df_ship.iloc[idx:].copy()
    df_temp['PathID'] = path_id
    df_new = df_new.append(df_temp,ignore_index=True)  #append whatever chunk contains the last row
    return df_new

In [ ]:
df_new2 = []
df_ship = df[df['UserID']==316005621]
df_new2 = find_paths(df_ship)
print(np.unique(df_new2['PathID']))
df_new2.head()

[1 2 3 4 5 6 7 8 9 10 11 12 13 14]


,UserID,ROT,SOG,Longitude,Latitude,TrueHeading,ReceivedTime,Time2,PathID
0,316005621,-127.0,0.1,-122.77156,49.23065,511.0,2013-10-22 01:05:24.510,1382403960,1
1,316005621,-127.0,0.1,-122.77149,49.23070,511.0,2013-10-22 01:05:35.540,1382403960,1
2,316005621,-127.0,0.1,-122.77143,49.23067,511.0,2013-10-22 01:06:06.293,1382403960,1
3,316005621,-127.0,0.1,-122.77141,49.23069,511.0,2013-10-22 01:06:27.153,1382403960,1
4,316005621,-127.0,0.1,-122.77143,49.23069,511.0,2013-10-22 01:06:41.230,1382403960,1


In [ ]:
#process all datasets in such manner
def initial_process_df(df):
    df = df.dropna(axis=0)
    df = df[df['UserID'].apply(lambda x: isinstance(x,(int,float)))]
    #extract only lat and lon needed
    mask = (df['Longitude']>-127)&(df['Longitude']<-120)
    mask = mask & (df['Latitude']>48)&(df['Latitude']<50)
    df = df[mask]
    
    # 6 minutes = 360 seconds
    df['Time2'] = df['ReceivedTime'].apply(lambda x: int(360*round(x.timestamp()/360)))
    
    #compute horizontal and vertical velocity
    # Therefore,
    # 1 knot = 1/60 in lat-lon degrees (along x or y coordinate) per hour
    #also note that the angle is with respect to north
    df['Speed_long'] = df['SOG']*df['TrueHeading'].apply(lambda x: np.cos(np.deg2rad(x+90)))/(60)
    df['Speed_lat'] = -df['SOG']*df['TrueHeading'].apply(lambda x: np.sin(np.deg2rad(x+90)))/(60)
    #now units of speed is in lat-lon degrees per hour
    
    
    #find distinct paths
    ships_list = list(np.unique(df['UserID']))
    col_names = df.columns.values.tolist()
    col_names.append('PathID')
    df_new = pd.DataFrame(columns=col_names) #here we store dataframe each of them is a part of unique path
    for ship in ships_list:
        #retrieve the data for the ship
        df_ship = df[df['UserID']==ship].sort_values(by=['Time2']).reset_index(drop=True)
        df_new = df_new.append(find_paths(df_ship),ignore_index=True)
    return df_new        

In [ ]:
#p = Pool(5)
delta_cur = pd.read_csv('../data/workshop-content18/3-snc/data/PMV_AIS_Deltaport_Current.txt', sep='\t',
    parse_dates=['ReceivedTime'],usecols=['UserID','ROT','SOG','Longitude','Latitude',
                                          'TrueHeading','ReceivedTime'])
df_new = initial_process_df(delta_cur)

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
df_new.head()

,UserID,ROT,SOG,Longitude,Latitude,TrueHeading,ReceivedTime,Time2,Speed_long,Speed_lat,PathID
0,209817000,0.0,11.1,-123.20550,48.40427,35.0,2013-10-28 17:46:25.417,1382982480,-0.106112,-0.151543,1
1,209817000,0.0,11.3,-123.19783,48.41012,35.0,2013-10-28 17:48:55.293,1382982480,-0.108024,-0.154274,1
2,209817000,0.0,11.3,-123.19692,48.41083,35.0,2013-10-28 17:49:13.887,1382982480,-0.108024,-0.154274,1
3,209817000,0.0,11.3,-123.19635,48.41127,34.0,2013-10-28 17:49:25.670,1382982480,-0.105315,-0.156135,1
4,209817000,0.0,11.3,-123.19525,48.41210,31.0,2013-10-28 17:49:45.623,1382982480,-0.096999,-0.161433,1


In [ ]:
df_new.to_csv("delta_cur_processed.csv")

In [ ]:
delta_hist = pd.read_csv('../data/workshop-content18/3-snc/data/PMV_AIS_Deltaport_History.txt',
                        sep='\t', parse_dates=['ReceivedTime'],
                         usecols=['UserID','ROT','SOG','Longitude','Latitude',
                                  'TrueHeading','ReceivedTime'])

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
df_new = initial_process_df(delta_hist)
df_new.to_csv("delta_hist_processed.csv")

In [ ]:
nw_cur = pd.read_csv('../data/workshop-content18/3-snc/data/PMV_AIS_NewWestminster_Current.txt',
                        sep='\t', parse_dates=['ReceivedTime'],
                        usecols=['UserID','ROT','SOG','Longitude','Latitude',
                                  'TrueHeading','ReceivedTime'])
df_new = initial_process_df(nw_cur)
df_new.to_csv("nw_cur_processed.csv")

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
nw_hist = pd.read_csv('../data/workshop-content18/3-snc/data/PMV_AIS_NewWestminster_History.txt',
                        sep='\t', parse_dates=['ReceivedTime'],
                         usecols=['UserID','ROT','SOG','Longitude','Latitude',
                                  'TrueHeading','ReceivedTime'])
df_new = initial_process_df(nw_hist)
df_new.to_csv("nw_hist_processed.csv")

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
